In [ ]:
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
import zipfile
import glob
import rasterio
import yaml
import yt
import yt.extensions.geotiff

### Download the data

In [ ]:
### format should be:
### user: <username>
### password: <password>
with open('sentinel.yaml', 'r') as f:
    info = yaml.load(f, Loader=yaml.FullLoader)

In [ ]:
api = SentinelAPI(
    user=info['user'],
    password=info['password'],
    api_url="https://scihub.copernicus.eu/dhus/",
)

geojson = read_geojson('footprint.geojson')
footprint = geojson_to_wkt(geojson)
date = ('NOW-30DAYS','NOW')
producttype = 'S2MSI2A'

products = api.query(footprint, date=date, producttype=producttype)

In [ ]:
product_df = api.to_dataframe(products)\
                .sort_values(['cloudcoverpercentage', 'ingestiondate'], ascending=[True, True])\
                .head(1)

In [ ]:
product_df[['cloudcoverpercentage', 'ingestiondate']] # hopefully we have an image with little cloud.

In [ ]:
download = api.download_all(product_df.index)

### Unzip and export 20m resolution bands to GeoTIFF

In [ ]:
s2zipfile = product_df.title.tolist()[0] + '.zip'
with zipfile.ZipFile(s2zipfile, 'r') as zip_ref:
    zip_ref.extractall('.')
s2ds = product_df.title.tolist()[0] + '.SAFE'

In [ ]:
s2r20m_files = sorted(glob.glob(s2ds + '/GRANULE/**/IMG_DATA/R20m/*_20m.jp2', recursive=True))

In [ ]:
bands = {}
ignored_bands = ['AOT', 'TCI', 'WVP', 'SCL'] # we just want the numbered bands
for f in s2r20m_files:
    label = f.split('_')[-2]
    if any([label == ignored_band for ignored_band in ignored_bands]): continue
    bands[label] = rasterio.open(f, driver='JP2OpenJPEG')

### Export multiband geotiff

In [ ]:
filename = product_df.title.tolist()[0] + '_20m.tif'
geotiff = rasterio.open(filename,'w',driver='Gtiff',
                         width=bands['B04'].width, height=bands['B04'].height,
                         count=len(bands.keys()),
                         crs=bands['B04'].crs,
                         transform=bands['B04'].transform,
                         dtype=bands['B04'].dtypes[0]
                         )
for i, label in enumerate(bands.keys()):
    geotiff.write(bands[label].read(1), i + 1)
geotiff.close()

In [ ]:
list(bands.keys()) # this order

#### Save band information to supplement file

In [ ]:
field_map = dict((str(i+1), band) for i, band in enumerate(bands))
data = {"field_type": "sentinel2", "field_map": field_map}
with open("sentinel2_fields.yaml", 'w') as f:
    yaml.dump(data, f)

We have produced a geotiff from the Sentinel 2 20m resolution data we downloaded. The next step could be to load it into yt and see what we get!